In [ ]:
!git clone https://github.com/mschoene/clvr_impl_starter.git

In [ ]:
%cd clvr_impl_starter

In [ ]:
!pip3 install virtualenv
!virtualenv -p $(which python3) ./venv
!source ./venv/bin/activate
# Install dependencies
!pip3 install -r requirements.txt

In [ ]:
from datetime import datetime
from sprites_datagen.moving_sprites import MovingSpriteDataset,DistractorTemplateMovingSpritesGenerator

from sprites_env.envs.sprites import SpritesEnv
from sprites_datagen.rewards import *

from general_utils import AttrDict
from torch.utils.data import Dataset, DataLoader,IterableDataset
from datasets import load_dataset
from general_utils import * #make_image_seq_strip
import numpy as np
import cv2
import sprites_datagen.moving_sprites 
from torch.autograd import Variable

from torchviz import make_dot
import matplotlib as plt

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

import torch.optim as optim

from models import ImageEncoder, ImageDecoder, AE, Autoencoder

In [ ]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
spec = AttrDict(
        resolution=64,
        max_seq_len=2, #30,
        max_speed=0.05,      # total image range [0, 1]
        obj_size=0.2,       # size of objects, full images is 1.0
        shapes_per_traj=20,      # number of shapes per trajectory
        rewards=[ZeroReward]
    )

n_conditioning_frames = 3
n_prediction_frames = 6 #TODO change to 25 or w/e
batch_size = 4096 #2048 #1024
n_batches = 200
n_samples = batch_size * n_batches

test_ds = MovingSpriteDataset(spec=spec, num_samples=n_samples)

dataloader = DataLoader(test_ds, batch_size=batch_size, num_workers=4)

input_channels = 3
output_size = 64

encoder = ImageEncoder(input_channels=3, output_size=64)
decoder = ImageDecoder(input_channels=64, output_size=64)

# Create an instance of Autoencoder
autoencoder = AE(encoder, decoder)
model = autoencoder

loss_fn = nn.MSELoss()
optimizer = optim.RAdam( model.parameters(), betas = (0.9, 0.999), weight_decay=0.001)

model.to(device)

In [ ]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for i_batch, sample_batched in enumerate(dataloader):
        inputs_pre = sample_batched.images[:, 0, ...].squeeze(1)
        labels = sample_batched.images[:, 0, ...].squeeze(1)

        inputs = inputs_pre.clone().detach().requires_grad_(True)
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        #inputs.retain_grad = True
        #inputs.requires_grad = True
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()     

        # Adjust learning weights
        optimizer.step()
        # Gather data and report
        running_loss += loss.item()
        if i_batch % n_batches == 0:
            last_loss = running_loss / float(batch_size) # loss per batch
            print('  batch {} loss: {}'.format(i_batch + 1, last_loss))
            #print(' l1 loss ', l1_lambda * l1_loss )
            tb_x = epoch_index * len(dataloader) + i_batch + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [ ]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(dataloader): #validation_loader):
            vinputs, vlabels = vdata.images[:, 0, ...].squeeze(1), vdata.images[:, 0, ...].squeeze(1)
            vinputs = vinputs.to(device)
            vlabels = vlabels.to(device)
            #print(vinputs.shape, vlabels.shape)
            voutputs = model(vinputs)
            #print(voutputs.shape)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1.)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        #torch.save(model.state_dict(), model_path)

    epoch_number += 1

In [ ]:
for i, vdata  in enumerate(dataloader):
    #this is the way to adopt the same plotting for the dataset images as for the traj images
    # it's a bit hacky but it works, so it's not stupid :,) also I don't need to change the code in multiple places so it's a win...
    # undo this/ (255./2) - 1.0
    print(vdata.images.shape )
    vinputs, vlabels = vdata.images[:, 0, ...], vdata.images[:, 0, ...].squeeze(1)
    print(vinputs.shape,vdata.images.shape )

    vinputs = vinputs.to(device)

    voutputs = model(vinputs)
    vinputs = vinputs.to('cpu')

    voutputs = voutputs.cpu().detach().numpy()
    img = make_image_seq_strip([ ((1+vinputs[None, :])*(255/2.))] ,n_logged_samples=5,sep_val=255.0).astype(np.uint8)
    cv2.imwrite("test_input.png", img[0].transpose(1, 2, 0))
    voutputs = voutputs[:5, ...] * 2 - 1
    #print(voutputs)
    print(vinputs.shape,vdata.images.shape )

    img = make_image_seq_strip([ ((1+voutputs[None, :])*(255/2.))] ,n_logged_samples=5,sep_val=255.0).astype(np.uint8)
    cv2.imwrite("test_output.png", img[0].transpose(1, 2, 0))
    if i ==0:
        break
